In [1]:
import pandas as pd
import numpy as np
import time
import importlib
import warnings
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA

import model_collection
from model_collection import *

pd.options.display.max_columns = None

In [2]:
fver = 'v10'
masterframe = loaddata_master('../Data/mf_USDJPY1440_'+fver+'.csv')

In [3]:
# Prepare Y
Rtp=1
Rsl=1
masterframe['y'] = -1
n = 5
i = 0
while i < len(masterframe) - n:   
    j = 1
    yy = False
    while j <= n:
        if not (masterframe.low.iloc[i+j] > masterframe.close.iloc[i]-Rsl*masterframe.atr14atr.iloc[i]):
        #if not (masterframe.low.iloc[i+j] > masterframe.low.iloc[i]-Rsl*masterframe.atr14atr.iloc[i]):
            yy = False
            break
        if (masterframe.high.iloc[i+j] > masterframe.close.iloc[i]+Rtp*masterframe.atr14atr.iloc[i]):
        #if (masterframe.high.iloc[i+j] > masterframe.high.iloc[i]+Rtp*masterframe.atr14atr.iloc[i]):
            yy = True
            break
        j = j + 1
            
    if yy == True:
        masterframe.iloc[i,masterframe.columns.get_loc('y')] = 1            
        #masterframe.iloc[i+1:i+j+1,masterframe.columns.get_loc('y')]=0      #nochain
        #i = i + j                                                           #nochain 
        i = i + 1   #chain
        
    else:
        masterframe.iloc[i,masterframe.columns.get_loc('y')] = 0
        i = i + 1

In [3]:
# Prepare Y
Rtp=0
Rsl=0
masterframe['y'] = -1
n = 1
i = 0
while i < len(masterframe) - n:   
    j = 1
    yy = False
    while j <= n:
        if (masterframe.high.iloc[i+j] > masterframe.high.iloc[i]):
            yy = True
            break
        j = j + 1
            
    if yy == True:
        masterframe.iloc[i,masterframe.columns.get_loc('y')] = 1            
        #masterframe.iloc[i+1:i+j+1,masterframe.columns.get_loc('y')]=0      #nochain
        #i = i + j                                                           #nochain 
        i = i + 1   #chain
        
    else:
        masterframe.iloc[i,masterframe.columns.get_loc('y')] = 0
        i = i + 1

In [23]:
masterframe[['fulldate','open','high','low','close','atr14atr','y']]

,fulldate,open,high,low,close,atr14atr,y
0,2003-05-04,118.940,119.046,118.461,118.603,NaN,0
1,2003-05-05,118.591,118.751,117.290,117.500,NaN,0
2,2003-05-06,117.456,117.830,116.052,116.303,NaN,0
3,2003-05-07,116.311,116.969,115.940,116.823,NaN,1
4,2003-05-08,116.835,117.612,116.794,117.151,NaN,0
5,2003-05-11,117.286,117.286,116.304,117.025,NaN,1
6,2003-05-12,117.053,117.528,116.338,116.561,NaN,0
7,2003-05-13,116.554,116.837,115.557,116.160,NaN,0
8,2003-05-14,116.173,116.525,115.282,116.515,NaN,1
9,2003-05-15,116.538,116.596,115.768,115.898,NaN,1


In [4]:
#pre-prepare data
orygframe = masterframe.copy()
masterframe = masterframe.drop(['volume'],1)
masterframe.dropna(inplace=True)

In [5]:
# split data
X_df = masterframe.iloc[:-1, 2:-1] 
y_df = masterframe.iloc[:-1, -1] 
featurenames = masterframe.iloc[:-1, 2:-1].columns.values

X = X_df.values
y = y_df.values
y = y.astype('int')
X = X.astype('float')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [6]:
#Scale
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_sc = scaler.transform(X_train)
X_test_sc = scaler.transform(X_test)

# Params

In [7]:
modeltype = {}
featcount = []
testone=False
# featsel = 'rf'
# featsel = 'svc'
# featsel = 'pca'
featsel = 'all'
featcount.append(5);modeltype[5]   = ['rf','svc','mlp']
featcount.append(10);modeltype[10] = ['rf','svc','mlp']
featcount.append(15);modeltype[15] = ['rf','svc','mlp']
featcount.append(20);modeltype[20] = ['rf','svc','mlp']
featcount.append(25);modeltype[25] = ['rf','svc','mlp']

In [ ]:
#feature_selection
importlib.reload(model_collection)
from model_collection import *

if featsel != 'all':
    for i in featcount:
        print('FEATSEL:'+featsel+str(i)+'________________________________________________________________________________________')

        if featsel == 'rf':
            select = RFE(RandomForestClassifier(n_estimators=100,random_state=2,n_jobs=1),n_features_to_select=i)
            select.fit(X_train, y_train)
            X_train_rfe= select.transform(X_train) 
            X_test_rfe= select.transform(X_test)
            X_train_sc_rfe= select.transform(X_train_sc)  
            X_test_sc_rfe= select.transform(X_test_sc)    

        if featsel == 'svc':
            select = RFE(SVC(kernel='linear')            ,n_features_to_select=i)
            select.fit(X_train_sc, y_train)
            X_train_rfe= select.transform(X_train) 
            X_test_rfe= select.transform(X_test)
            X_train_sc_rfe= select.transform(X_train_sc)  
            X_test_sc_rfe= select.transform(X_test_sc)  

        if featsel == 'pca':
            select = PCA(n_components=i, whiten=True, random_state=2)
            select.fit(X_train)
            X_train_rfe= select.transform(X_train) 
            X_test_rfe= select.transform(X_test)
            X_train_sc_rfe= X_train_rfe
            X_test_sc_rfe= X_test_rfe

    #     select = PCA(n_components=i, whiten=False, random_state=2)
    #     select.fit(X_train_sc)
    #     X_train_rfe= select.transform(X_train) 
    #     X_test_rfe= select.transform(X_test)
    #     X_train_sc_rfe= select.transform(X_train_sc)
    #     X_test_sc_rfe= select.transform(X_test_sc)
    #     featsel = 'pca_nw'

        if testone == True:
            featsel = 'temp_'+featsel

        # visualize the selected features:
        #mask = select.get_support()
        #plt.matshow(mask.reshape(1, -1), cmap='gray_r')
        #plt.xlabel("Sample index")
        #print(X_df.iloc[:2,mask])
        #print("Test score: {:.3f}".format(select.score(X_test_sc, y_test)))
        #print("Test score: {:.3f}".format(select.score(X_test, y_test)))

        #lin_resdf = ExamineLogisticRegression(orygframe,X_test[:,0],X_train_rfe, y_train,X_test_rfe, y_test,featurenames,testone=False,plot=False)
        #lin_resdf.to_csv(sep=';',path_or_buf='Resu/'+fver+'_'+featsel+str(i)+'_LogisticRegression'+str(int(time.time()))+'.csv',date_format="%Y-%m-%d",index = False)

        #lin_resdf = ExamineLinearSVC(orygframe,X_test[:,0],X_train_rfe, y_train,X_test_rfe, y_test,featurenames,testone=False,plot=False)
        #lin_resdf.to_csv(sep=';',path_or_buf='Resu/'+fver+'_'+featsel+str(i)+'_LinearSVC'+str(int(time.time()))+'.csv',date_format="%Y-%m-%d",index = False)


        if 'rf' in modeltype[i]:
            print('FEATSEL:'+featsel+str(i)+'_model_rf___________________________________________________________________________')
            forest_resdf = ExamineRandomForest(orygframe,X_test[:,0],X_train_rfe, y_train,X_test_rfe, y_test,featurenames,testone=testone,plot=False,automaxfeat=True)
            forest_resdf.to_csv(sep=';',path_or_buf='../Resu/'+fver+'_'+featsel+str(i)+'_RandomForest'+str(int(time.time()))+'.csv',date_format="%Y-%m-%d",index = False)

        if 'svc' in modeltype[i]:
            print('FEATSEL:'+featsel+str(i)+'_model_svc___________________________________________________________________________')
            svc_resdf = ExamineSVC(orygframe,X_test[:,0],X_train_sc_rfe, y_train,X_test_sc_rfe, y_test,featurenames,testone=testone,plot=False)
            svc_resdf.to_csv(sep=';',path_or_buf='../Resu/'+fver+'_'+featsel+str(i)+'_SVC'+str(int(time.time()))+'.csv',date_format="%Y-%m-%d",index = False)

        if 'mlp' in modeltype[i]:
            print('FEATSEL:'+featsel+str(i)+'_model_mlp___________________________________________________________________________')
            mlp_resdf = ExamineMLP(orygframe,X_test[:,0],X_train_sc_rfe, y_train,X_test_sc_rfe, y_test,featurenames,testone=testone,plot=False)
            mlp_resdf.to_csv(sep=';',path_or_buf='../Resu/'+fver+'_'+featsel+str(i)+'_MLP'+str(int(time.time()))+'.csv',date_format="%Y-%m-%d",index = False)

    print('FEATSEL________________finished________________________________________________________________________________')


FEATSEL:rf15________________________________________________________________________________________
FEATSEL:rf15_model_svc___________________________________________________________________________
2019-10-16 09:25:47.939979 rbf 0.001 auto_deprecated -1 2
train:       51.8 | test:       53.4 | cal:       37.4 | proctime: 0:00:02.245419
2019-10-16 09:25:51.588353 rbf 0.001 auto -1 2
train:       51.8 | test:       53.4 | cal:       37.4 | proctime: 0:00:02.306346
2019-10-16 09:25:55.326739 rbf 0.001 0.0001 -1 2
train:       51.8 | test:       53.4 | cal:       37.4 | proctime: 0:00:02.183785
2019-10-16 09:25:58.820253 rbf 0.001 0.001 -1 2
train:       51.8 | test:       53.4 | cal:       37.5 | proctime: 0:00:02.194197
2019-10-16 09:26:02.415197 rbf 0.001 0.01 -1 2
train:       51.8 | test:       53.4 | cal:       40.9 | proctime: 0:00:02.143079
2019-10-16 09:26:05.854219 rbf 0.001 0.1 -1 2
train:       51.8 | test:       53.4 | cal:       37.5 | proctime: 0:00:02.204532
2019-10-16 09:

train:       51.8 | test:       53.4 | cal:       36.9 | proctime: 0:00:06.724787
2019-10-16 09:30:19.283224 linear 0.001 10 -1 2
train:       51.8 | test:       53.4 | cal:       36.9 | proctime: 0:00:06.741087
2019-10-16 09:30:26.749874 linear 0.01 auto_deprecated -1 2
train:       61.9 | test:       61.7 | cal:       29.8 | proctime: 0:00:06.884882
2019-10-16 09:30:34.336359 linear 0.01 auto -1 2
train:       61.9 | test:       61.7 | cal:       29.8 | proctime: 0:00:07.408567
2019-10-16 09:30:42.459775 linear 0.01 0.0001 -1 2
train:       61.9 | test:       61.7 | cal:       29.8 | proctime: 0:00:06.599687
2019-10-16 09:30:49.796172 linear 0.01 0.001 -1 2
train:       61.9 | test:       61.7 | cal:       29.8 | proctime: 0:00:07.170368
2019-10-16 09:30:57.909789 linear 0.01 0.01 -1 2
train:       61.9 | test:       61.7 | cal:       29.8 | proctime: 0:00:06.737350
2019-10-16 09:31:05.399664 linear 0.01 0.1 -1 2
train:       61.9 | test:       61.7 | cal:       29.8 | proctime: 0:00

train:       51.8 | test:       53.4 | cal:        0.0 | proctime: 0:00:06.967198
2019-10-16 09:38:46.616287 poly 0.001 0.001 -1 4
train:       51.8 | test:       53.4 | cal:        0.0 | proctime: 0:00:06.671672
2019-10-16 09:38:54.169357 poly 0.001 0.001 -1 5
train:       51.8 | test:       53.4 | cal:        0.0 | proctime: 0:00:06.680805
2019-10-16 09:39:01.736278 poly 0.001 0.01 -1 2
train:       51.8 | test:       53.4 | cal:       30.7 | proctime: 0:00:06.600571
2019-10-16 09:39:09.184946 poly 0.001 0.01 -1 3
train:       51.8 | test:       53.4 | cal:       23.6 | proctime: 0:00:06.569970
2019-10-16 09:39:16.603477 poly 0.001 0.01 -1 4
train:       51.8 | test:       53.4 | cal:        0.0 | proctime: 0:00:06.772257
2019-10-16 09:39:24.262247 poly 0.001 0.01 -1 5
train:       51.8 | test:       53.4 | cal:        0.0 | proctime: 0:00:06.613680
2019-10-16 09:39:31.787793 poly 0.001 0.1 -1 2
train:       51.8 | test:       53.4 | cal:       31.9 | proctime: 0:00:06.892457
2019-10

train:       51.8 | test:       53.4 | cal:       23.6 | proctime: 0:00:08.532901
2019-10-16 09:58:30.111363 poly 0.1 0.001 -1 4
train:       51.8 | test:       53.4 | cal:        0.0 | proctime: 0:00:07.306352
2019-10-16 09:58:38.475584 poly 0.1 0.001 -1 5
train:       51.8 | test:       53.4 | cal:        0.0 | proctime: 0:00:07.756353
2019-10-16 09:58:47.170615 poly 0.1 0.01 -1 2
train:       51.8 | test:       53.4 | cal:       31.9 | proctime: 0:00:07.479399
2019-10-16 09:58:55.552780 poly 0.1 0.01 -1 3
train:       51.8 | test:       53.4 | cal:       23.6 | proctime: 0:00:07.325983
2019-10-16 09:59:03.828561 poly 0.1 0.01 -1 4
train:       51.8 | test:       53.4 | cal:       16.5 | proctime: 0:00:07.825906
2019-10-16 09:59:12.760773 poly 0.1 0.01 -1 5
train:       51.8 | test:       53.4 | cal:       12.5 | proctime: 0:00:07.838981
2019-10-16 09:59:21.573105 poly 0.1 0.1 -1 2
train:       62.5 | test:       60.9 | cal:       27.5 | proctime: 0:00:07.411091
2019-10-16 09:59:29.8

train:       51.8 | test:       53.4 | cal:        0.0 | proctime: 0:00:08.792315
2019-10-16 10:31:33.309207 poly 10 0.01 -1 2
train:       62.5 | test:       60.9 | cal:       27.5 | proctime: 0:00:08.537092
2019-10-16 10:31:42.948491 poly 10 0.01 -1 3
train:       51.8 | test:       53.4 | cal:       24.0 | proctime: 0:00:08.668984
2019-10-16 10:31:52.741916 poly 10 0.01 -1 4
train:       51.8 | test:       53.4 | cal:       16.5 | proctime: 0:00:08.759205
2019-10-16 10:32:02.653612 poly 10 0.01 -1 5
train:       51.8 | test:       53.4 | cal:       12.5 | proctime: 0:00:08.364116
2019-10-16 10:32:12.170646 poly 10 0.1 -1 2
train:       69.1 | test:       68.6 | cal:       41.3 | proctime: 0:00:08.585488
2019-10-16 10:32:21.619519 poly 10 0.1 -1 3
train:       67.6 | test:       68.5 | cal:       39.3 | proctime: 0:00:08.089947
2019-10-16 10:32:30.657371 poly 10 0.1 -1 4
train:       64.9 | test:       64.6 | cal:       33.9 | proctime: 0:00:08.405029
2019-10-16 10:32:40.076103 poly 

In [8]:
#no feature selection
importlib.reload(model_collection)
from model_collection import *

if featsel == 'all':

    # featsel = 'all_LogisticRegression' if testone == False else 'temp_all_LogisticRegression'
    # lin_resdf = ExamineLogisticRegression(orygframe,X_test[:,0],X_train, y_train,X_test, y_test,featurenames,testone=testone,plot=False)
    # lin_resdf.to_csv(sep=';',path_or_buf='Resu/'+fver+'_'+featsel+str(int(time.time()))+'.csv',date_format="%Y-%m-%d",index = False)

    # featsel = 'all_LinearSVC' if testone == False else 'temp_all_LinearSVC'
    # lin_resdf = ExamineLinearSVC(orygframe,X_test[:,0],X_train, y_train,X_test, y_test,featurenames,testone=testone,plot=False)
    # lin_resdf.to_csv(sep=';',path_or_buf='Resu/'+fver+'_'+featsel+str(int(time.time()))+'.csv',date_format="%Y-%m-%d",index = False)

    print('ALL_____________rf___________________________________________________________________________')
    featsel = 'all_RandomForest' if testone == False else 'temp_all_RandomForest'
    forest_resdf = ExamineRandomForest(orygframe,X_test[:,0],X_train, y_train,X_test, y_test,featurenames,testone=testone,plot=False,automaxfeat=False)
    forest_resdf.to_csv(sep=';',path_or_buf='../Resu/'+fver+'_'+featsel+str(int(time.time()))+'.csv',date_format="%Y-%m-%d",index = False)

    print('ALL_____________svc___________________________________________________________________________')
    featsel = 'all_SVC' if testone == False else 'temp_all_SVC'
    svc_resdf = ExamineSVC(orygframe,X_test[:,0],X_train_sc, y_train,X_test_sc, y_test,featurenames,testone=testone,plot=False)
    svc_resdf.to_csv(sep=';',path_or_buf='../Resu/'+fver+'_'+featsel+str(int(time.time()))+'.csv',date_format="%Y-%m-%d",index = False)

    print('ALL_____________mlp___________________________________________________________________________')
    featsel = 'all_MLP' if testone == False else 'temp_all_MLP'
    mlp_resdf = ExamineMLP(orygframe,X_test[:,0],X_train_sc, y_train,X_test_sc, y_test,featurenames,testone=testone,plot=False)
    mlp_resdf.to_csv(sep=';',path_or_buf='../Resu/'+fver+'_'+featsel+str(int(time.time()))+'.csv',date_format="%Y-%m-%d",index = False)

    print('ALL________________finished________________________________________________________________________________')

ALL_____________rf___________________________________________________________________________
2019-10-18 08:57:02.803039 auto 100 5 5
train:       68.9 | test:       66.7 | cal:       18.1 | proctime: 0:00:01.952855
2019-10-18 08:57:05.182763 auto 100 5 10
train:       71.0 | test:       66.8 | cal:       25.1 | proctime: 0:00:02.258994
2019-10-18 08:57:07.848707 auto 100 5 50
train:       74.1 | test:       67.2 | cal:       27.1 | proctime: 0:00:03.199464
2019-10-18 08:57:12.009651 auto 100 5 100
train:       74.1 | test:       67.2 | cal:       27.1 | proctime: 0:00:03.032904
2019-10-18 08:57:15.590099 auto 100 5 300
train:       74.1 | test:       67.2 | cal:       27.1 | proctime: 0:00:03.575498
2019-10-18 08:57:19.600469 auto 100 30 5
train:       68.9 | test:       66.7 | cal:       18.1 | proctime: 0:00:02.007634
2019-10-18 08:57:22.010065 auto 100 30 10
train:       70.8 | test:       66.4 | cal:       25.3 | proctime: 0:00:02.415550
2019-10-18 08:57:24.831566 auto 100 30 50
t

train:       69.0 | test:       66.0 | cal:       18.4 | proctime: 0:00:15.867769
2019-10-18 09:11:28.620709 auto 700 100 10
train:       70.7 | test:       66.1 | cal:       26.1 | proctime: 0:00:20.134457
2019-10-18 09:11:50.513494 auto 700 100 50
train:       81.8 | test:       67.5 | cal:       36.0 | proctime: 0:00:31.174098
2019-10-18 09:12:24.067266 auto 700 100 100
train:       90.9 | test:       67.2 | cal:       37.7 | proctime: 0:00:30.649496
2019-10-18 09:12:59.158983 auto 700 100 300
train:       99.6 | test:       66.0 | cal:       35.7 | proctime: 0:00:40.430454
2019-10-18 09:13:42.828824 auto 700 300 5
train:       69.0 | test:       66.0 | cal:       18.4 | proctime: 0:00:12.831877
2019-10-18 09:13:57.159716 auto 700 300 10
train:       70.7 | test:       66.1 | cal:       26.1 | proctime: 0:00:16.283699
2019-10-18 09:14:15.199747 auto 700 300 50
train:       81.8 | test:       67.5 | cal:       36.0 | proctime: 0:00:30.169769
2019-10-18 09:14:47.979578 auto 700 300 10

train:       70.1 | test:       66.1 | cal:       12.8 | proctime: 0:00:05.552231
2019-10-18 09:33:29.259837 log2 400 30 50
train:       81.3 | test:       67.1 | cal:       24.2 | proctime: 0:00:06.462802
2019-10-18 09:33:37.201706 log2 400 30 100
train:       90.2 | test:       66.7 | cal:       28.0 | proctime: 0:00:07.559884
2019-10-18 09:33:47.959110 log2 400 30 300
train:       99.5 | test:       65.3 | cal:       30.4 | proctime: 0:00:08.769636
2019-10-18 09:33:58.771313 log2 400 70 5
train:       67.5 | test:       65.7 | cal:        8.0 | proctime: 0:00:03.479741
2019-10-18 09:34:03.306248 log2 400 70 10
train:       70.1 | test:       66.1 | cal:       12.8 | proctime: 0:00:04.029278
2019-10-18 09:34:08.495440 log2 400 70 50
train:       81.3 | test:       67.1 | cal:       24.2 | proctime: 0:00:06.316192
2019-10-18 09:34:16.274742 log2 400 70 100
train:       90.2 | test:       66.7 | cal:       28.0 | proctime: 0:00:07.548912
2019-10-18 09:34:25.767481 log2 400 70 300
train

train:       81.4 | test:       66.6 | cal:       24.1 | proctime: 0:00:18.842812
2019-10-18 09:48:33.200818 log2 900 300 100
train:       90.5 | test:       66.8 | cal:       27.8 | proctime: 0:00:26.128508
2019-10-18 09:49:04.241253 log2 900 300 300
train:       99.5 | test:       65.7 | cal:       29.2 | proctime: 0:00:20.797640
2019-10-18 09:49:29.879010 None 100 5 5
train:       69.6 | test:       65.7 | cal:       38.1 | proctime: 0:00:57.235671
2019-10-18 09:50:27.481690 None 100 5 10
train:       72.0 | test:       67.2 | cal:       42.0 | proctime: 0:00:58.027599
2019-10-18 09:51:26.135624 None 100 5 50
train:       75.8 | test:       67.6 | cal:       41.8 | proctime: 0:01:06.878961
2019-10-18 09:52:33.590081 None 100 5 100
train:       75.8 | test:       67.6 | cal:       41.8 | proctime: 0:01:13.076443
2019-10-18 09:53:47.220054 None 100 5 300
train:       75.8 | test:       67.6 | cal:       41.8 | proctime: 0:01:08.631296
2019-10-18 09:54:56.487656 None 100 30 5
train:   

train:       94.7 | test:       67.3 | cal:       46.6 | proctime: 0:12:49.306007
2019-10-18 15:07:38.450175 None 700 70 300
train:      100.0 | test:       66.8 | cal:       39.2 | proctime: 0:21:39.539487
2019-10-18 15:29:21.884269 None 700 100 5
train:       69.9 | test:       65.7 | cal:       37.3 | proctime: 0:05:20.487320
2019-10-18 15:34:44.252570 None 700 100 10
train:       71.9 | test:       66.6 | cal:       42.9 | proctime: 0:07:00.381400
2019-10-18 15:41:46.576809 None 700 100 50
train:       85.6 | test:       67.8 | cal:       46.1 | proctime: 0:10:39.845635
2019-10-18 15:52:28.913794 None 700 100 100
train:       94.7 | test:       67.3 | cal:       46.6 | proctime: 0:13:01.075446
2019-10-18 16:05:32.999206 None 700 100 300
train:      100.0 | test:       66.8 | cal:       39.2 | proctime: 0:17:34.815966
2019-10-18 16:23:11.070481 None 700 300 5
train:       69.9 | test:       65.7 | cal:       37.3 | proctime: 0:04:51.909567
2019-10-18 16:28:04.486026 None 700 300 10


...

train:       88.9 | test:       62.1 | cal:       91.4 | proctime: 0:00:09.151574
2019-10-19 11:11:45.470248 lbfgs [400, 400, 400] tanh 0.0001 100
train:       73.8 | test:       65.2 | cal:       56.7 | proctime: 0:00:00.983375
2019-10-19 11:11:46.679023 lbfgs [400, 400, 400] tanh 0.0001 1000
train:       89.3 | test:       65.0 | cal:       89.3 | proctime: 0:00:09.144588
2019-10-19 11:11:56.041055 lbfgs [400, 400, 400] tanh 0.01 100
train:       72.5 | test:       65.2 | cal:       56.9 | proctime: 0:00:00.902565
2019-10-19 11:11:57.161038 lbfgs [400, 400, 400] tanh 0.01 1000
train:       88.2 | test:       63.4 | cal:       78.5 | proctime: 0:00:09.161571
2019-10-19 11:12:06.535015 lbfgs [400, 400, 400] tanh 0.1 100
train:       73.3 | test:       65.0 | cal:       54.1 | proctime: 0:00:00.893612
2019-10-19 11:12:07.753761 lbfgs [400, 400, 400] tanh 0.1 1000
train:       92.6 | test:       62.3 | cal:       85.1 | proctime: 0:00:09.256284
2019-10-19 11:12:17.236442 lbfgs [400, 400,

train:       92.6 | test:       62.3 | cal:       85.1 | proctime: 0:00:09.492975
2019-10-19 11:16:32.719945 lbfgs [800, 800, 800] tanh 1 100
train:       73.0 | test:       66.4 | cal:       53.1 | proctime: 0:00:00.891648
2019-10-19 11:16:33.835032 lbfgs [800, 800, 800] tanh 1 1000
train:       95.6 | test:       61.1 | cal:       91.5 | proctime: 0:00:09.397206
2019-10-19 11:16:43.440698 lbfgs [800, 800, 800] relu 1e-05 100
train:       64.2 | test:       64.0 | cal:       28.9 | proctime: 0:00:00.267289
2019-10-19 11:16:43.928406 lbfgs [800, 800, 800] relu 1e-05 1000
train:       64.2 | test:       64.0 | cal:       28.9 | proctime: 0:00:00.282258
2019-10-19 11:16:44.420108 lbfgs [800, 800, 800] relu 0.0001 100
train:       70.8 | test:       68.1 | cal:       50.4 | proctime: 0:00:00.922570
2019-10-19 11:16:45.559134 lbfgs [800, 800, 800] relu 0.0001 1000
train:       80.3 | test:       66.1 | cal:       72.6 | proctime: 0:00:09.223661
2019-10-19 11:16:55.004213 lbfgs [800, 800, 8

train:       80.3 | test:       66.1 | cal:       72.6 | proctime: 0:00:09.218688
2019-10-19 11:21:10.404917 lbfgs [10, 10, 10, 10] relu 0.01 100
train:       71.3 | test:       66.5 | cal:       47.3 | proctime: 0:00:00.974431
2019-10-19 11:21:11.583808 lbfgs [10, 10, 10, 10] relu 0.01 1000
train:       79.3 | test:       66.0 | cal:       66.9 | proctime: 0:00:09.171819
2019-10-19 11:21:20.980031 lbfgs [10, 10, 10, 10] relu 0.1 100
train:       70.4 | test:       68.4 | cal:       45.4 | proctime: 0:00:00.972440
2019-10-19 11:21:22.166903 lbfgs [10, 10, 10, 10] relu 0.1 1000
train:       77.4 | test:       67.2 | cal:       65.0 | proctime: 0:00:09.484990
2019-10-19 11:21:31.883285 lbfgs [10, 10, 10, 10] relu 1 100
train:       70.4 | test:       68.2 | cal:       50.5 | proctime: 0:00:00.967475
2019-10-19 11:21:33.083123 lbfgs [10, 10, 10, 10] relu 1 1000
train:       77.0 | test:       68.2 | cal:       63.8 | proctime: 0:00:09.336376
2019-10-19 11:21:42.626957 lbfgs [100, 100, 100

train:       70.4 | test:       68.2 | cal:       50.5 | proctime: 0:00:00.950488
2019-10-19 11:25:56.122378 lbfgs [100, 100, 100, 100] relu 1 1000
train:       77.0 | test:       68.2 | cal:       63.8 | proctime: 0:00:09.004274
2019-10-19 11:26:05.344052 lbfgs [400, 400, 400, 400] tanh 1e-05 100
train:       73.2 | test:       66.2 | cal:       50.9 | proctime: 0:00:01.026291
2019-10-19 11:26:06.588769 lbfgs [400, 400, 400, 400] tanh 1e-05 1000
train:       88.9 | test:       62.1 | cal:       91.4 | proctime: 0:00:09.155845
2019-10-19 11:26:15.966060 lbfgs [400, 400, 400, 400] tanh 0.0001 100
train:       73.8 | test:       65.2 | cal:       56.7 | proctime: 0:00:01.000333
2019-10-19 11:26:17.181825 lbfgs [400, 400, 400, 400] tanh 0.0001 1000
train:       89.3 | test:       65.0 | cal:       89.3 | proctime: 0:00:09.057105
2019-10-19 11:26:26.463339 lbfgs [400, 400, 400, 400] tanh 0.01 100
train:       72.5 | test:       65.2 | cal:       56.9 | proctime: 0:00:00.952487
2019-10-19 1

train:       73.8 | test:       65.2 | cal:       56.7 | proctime: 0:00:00.887686
2019-10-19 11:30:33.594306 lbfgs [800, 800, 800, 800] tanh 0.0001 1000
train:       89.3 | test:       65.0 | cal:       89.3 | proctime: 0:00:09.571708
2019-10-19 11:30:43.395416 lbfgs [800, 800, 800, 800] tanh 0.01 100
train:       72.5 | test:       65.2 | cal:       56.9 | proctime: 0:00:00.889647
2019-10-19 11:30:44.562334 lbfgs [800, 800, 800, 800] tanh 0.01 1000
train:       88.2 | test:       63.4 | cal:       78.5 | proctime: 0:00:09.284492
2019-10-19 11:30:54.060292 lbfgs [800, 800, 800, 800] tanh 0.1 100
train:       73.3 | test:       65.0 | cal:       54.1 | proctime: 0:00:00.893616
2019-10-19 11:30:55.250124 lbfgs [800, 800, 800, 800] tanh 0.1 1000
train:       92.6 | test:       62.3 | cal:       85.1 | proctime: 0:00:09.238620
2019-10-19 11:31:04.711155 lbfgs [800, 800, 800, 800] tanh 1 100
train:       73.0 | test:       66.4 | cal:       53.1 | proctime: 0:00:00.888651
2019-10-19 11:31:0